# run a set of parameters through the SMFF model

### paths

In [1]:
import os, socket

if socket.gethostname() == 'kumo':
    experimentpath = os.path.normpath('/home/michael/datac/experiments/20150612/')
else:
    datapath = os.path.normpath('/Users/michael/coding/RIKEN/data/150612/')
    
smfpath = os.path.join(experimentpath, 'smf')
ANALYSIS_NAME = 'tests'
outpath = os.path.join(smfpath, ANALYSIS_NAME)
if not os.path.exists(outpath):
    os.makedirs(outpath)

datapath = os.path.join(experimentpath, 'data')
datafile = os.path.join(datapath, 'Image001_Registered_16bit_cutout.tif')

### the parameters

In [2]:
# model init parameters
components = ((2, 3, 40),(8, 3, 20),(5, 2, 20))  # (num components, sigma gaussian, window size multiplier,)
init_its = 5

# smff parameters
iterations = 5
morph_mod = 2

# data
fs = 15.

# processing
N_JOBS = 20 # two less than cpu count

### running the code

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'none'
plt.rcParams['figure.figsize'] = (5,5)

import numpy as np
import sklearn

In [4]:
import neuralyzer
from neuralyzer.im.smff import model, _init_model, nbplot

[ 2015-07-30 20:10:51 ] [ log ] [ INFO ] : NEURALYZER LOGGER CREATED
[ 2015-07-30 20:10:51 ] [ log ] [ DEBUG ] : stdoutloglevel: DEBUG


In [5]:
# load data
data = neuralyzer.get_data(datafile, library='tifffile')

[ 2015-07-30 20:10:52 ] [ data_handler ] [ DEBUG ] : root_path set to /home/michael/lib/neuralyzer/notebooks/dev
[ 2015-07-30 20:10:52 ] [ data_handler ] [ DEBUG ] : loaded data from cache file: /home/michael/datac/experiments/20150612/data/Image001_Registered_16bit_cutout.tif.hdf5


In [6]:
ds = data.shape
data = data.reshape((data.shape[0], data.shape[1]*data.shape[2])).T
data.shape

(65536, 3000)

In [ ]:
Ainit, Cinit, binit, finit = _init_model.greedy(data, components=components, njobs=-1)

[ 2015-07-30 20:10:53 ] [ _init_model ] [ INFO ] : Initializing SMFF model with greedy algorithm.
[ 2015-07-30 20:10:53 ] [ _init_model ] [ DEBUG ] : copying data ..
[ 2015-07-30 20:10:58 ] [ _init_model ] [ INFO ] : subtracting background
[ 2015-07-30 20:11:05 ] [ _init_model ] [ INFO ] : Finding 2 components with sigma 3 and window size 121
[ 2015-07-30 20:11:05 ] [ _init_model ] [ DEBUG ] : component 1 / 2


In [ ]:
fig, ax = nbplot.plot_spatial_components(Ainit)
fig.savefig(os.path.join(outpath, 'init_spatial_components.png'))

In [ ]:
fig, ax = nbplot.plot_temporal_components(Cinit, fs)
fig.savefig(os.path.join(outpath, 'init_temporal_components.png'))

In [ ]:
smf = model.SMFF(A=Ainit, C=Cinit, b=binit, f=finit)
smf.fit_model(data, re_init=True, morph_mod=morph_mod, max_num_iterations=iterations, njobs=N_JOBS, maxiter=300)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(smf._avg_abs_res)
fig.savefig(os.path.join(outpath, 'residuals.png'))

In [ ]:
fig, ax = nbplot.plot_spatial_components(smf.A_)
fig.savefig(os.path.join(outpath, 'spatial_components.png'))

In [ ]:
fig, ax = nbplot.plot_temporal_components(smf.C_, 15.)
fig.savefig(os.path.join(outpath, 'temporal_components.png'))

In [ ]:
fig, ax = plt.subplots(ncols=1,nrows=1)
ax.grid(False)
imp = ax.imshow(smf.b_.reshape(ds[1],ds[2]),cmap='gray')
fig.savefig(os.path.join(outpath, 'background.png'))

In [ ]:
fig, ax = nbplot.plot_temporal_components(smf.f_, 15.)
fig.savefig(os.path.join(outpath, 'background_trace.png'))

In [ ]:
fig, ax = nbplot.plot_correlation_matrix(smf.C_)
fig.savefig(os.path.join(outpath, 'correlation_matrix.png'))